
Transformer架构基于Attendtion，出自著名论文[《Attention is all your need》](https://proceedings.neurips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)，以下是论文中描述Transformer的架构图，整体包含左侧编码和右侧解码

![图片](https://i-blog.csdnimg.cn/blog_migrate/9b4be748d671fd91c6853ca02ea841c7.png#pic_center)

以左侧编码为例说明，从下到上，首先输入是一串token，以字符为例



input embedding: 由于计算机无法理解token所表达的意思，需要将token转化成d维的向量表达token的含义， 用向量的维度作为词的属性表示。
```
下面是原文介绍
3.4 Embeddings and Softmax
Similarly to other sequence transduction models, we use learned embeddings to convert the input
tokens and output tokens to vectors of dimension dmodel. We also use the usual learned linear transformation and softmax function to convert the decoder output to predicted next-token probabilities. In
our model, we share the same weight matrix between the two embedding layers and the pre-softmax
linear transformation, similar to [24]. In the embedding layers, we multiply those weights by √
dmodel.
```
用pytorch做一个简易实现：

In [25]:
import torch 
import torch.nn as nn
import jieba
import numpy as np

class Tokenizer:
    def tokenize(self, text):
        return list(jieba.cut(text))
class EmbeddingLayer(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.d_model = embedding_dim
    def forward(self, tokens):
        return self.embedding(tokens) * self.d_model ** 0.5

text = '我爱北京天安门'
d_model = 10
# 先分词
tokenizer = Tokenizer()
tokens = tokenizer.tokenize(text)
print('分词结果：',tokens)
# 构建onehot词汇表,这里简单处理，直接用input的词作为词汇表
vocab = list(set(tokens))
vocab_size = len(vocab)
# 构建embedding层
embedding = EmbeddingLayer(vocab_size, d_model)
# token转成序列
input_ids = [vocab.index(token) for token in tokens]
inputs = torch.tensor(input_ids)
# 输出embedding
outputs = embedding(inputs)
print(outputs.shape)



分词结果： ['我', '爱', '北京', '天安门']
torch.Size([4, 10])


Q：为什么用Transformer中input Embedding的前向传播需要*√dmodel？

A：如果不乘，因为embedding通常用标准正态分布初始化，嵌入值如果太小会导致注意力分数也小，梯度变小，使模型无法有效学习。也不能乘的过大，导致嵌入值过大导致注意力分数过大，梯度消失，收敛困难。


position encoding: 位置编码，经过前一步后每个token有了自己的向量表示，但是人类理解的token是有顺序的，比如“狗是人养的”和“人是狗养的”明显不是一个意思。而计算机如果只基于token理解句子将认为它们是一个意思，所以我们需要加入位置编码，让程序了解到token顺序。

详看原文中3.5关于position encoding的描述，tranformer中的位置编码实现公式如下：

$$PE_{(pos,2i)} = \sin\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right)$$
$$PE_{(pos,2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right)$$

实现不局限与当前，也可以做学习式的，下面用原文中非学习式的三角函数做个简要的实现

In [ ]:
import math
#接上面的代码
class PositionEncoding(nn.Module):
    def __init__(self, embedding, token_size, d_model):
        super().__init__()
        self.pe = torch.zeros((token_size, d_model))
        self.normalPositionEncoding(token_size, d_model)
    def normalPositionEncoding(self, token_size, d_model):
        pe = self.pe
        # 这里需要抛弃for，用矩阵思维
        position = torch.arange(token_size).unsqueeze(1)
        div_term = 10000 ** -(torch.arange(0, d_model, 2) / d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

    def forward(self, input):
        return input + self.pe

position_encoding = PositionEncoding(embedding, vocab_size, d_model)
position_encoding_outputs = position_encoding(outputs)
print(position_encoding_outputs.shape)

torch.Size([4, 10])


Q: 为什么Transformer的Position Encoding要用正弦、余弦？

A: 正弦、余弦有几个方面好处
1. 感知相对位置：对于固定的偏移k，可以得到PE(pos+k) = M(k) * PE(pos), M(k)是基于k变量的变换矩阵[详细证明可见相关证明-PositionEncoding.ipynb]。不同的相对距离k，M(k)矩阵不一样
2. 多尺度感知：同时捕获不同粒度的时间/位置关系, 低维高频率叠加细粒度位置细节，高维低频率迭代粗粒度位置细节（段落、文档结构）
3. 唯一性：我们当前的d_model一般小于10000, i = d_model//2, 三角函数的波长在2PI， 而$\frac{pos+k1}{10000^{2(i+k2)/d_{\text{model}}}}$= (2PI or PI/2) + $\frac{pos}{10000^{2(i)/d_{\text{model}}}}$ 涉及无理数，在有限数字表示内（非天文数字）只有唯一的位置表示,不会重复。

Q: 为什么要用sin和cos交替，直接用sin或者直接用cos行不行？

A： 不行，如果直接用sin或者直接用cos，模型将无法通过线性变换得到PE(pos+k) = M(k) * PE(pos)[详细证明可见相关证明-PositionEncoding.ipynb].假设只用sign，令w_i = 1/(10000^(2i/d_model))。
PE(pos+k, 2i) = sin((pos+k) * w_i) = sin(pos*w_i + k*w_i) 
              = sin(pos*w_i)cos(k*w_i) + cos(pos*w_i)sin(k*w_i) = PE(pos, 2i)cos(k*w_i) + cos(pos*w_i)sin(k*w_i) 

cos(pos*w_i)是个未知数

Q: 分母为什么是$10000^{2i/d_{\text{model}}}$?

A：10000是个超参数，用于确保位置相对维度的衰减速率。如果值过小，频率就会增大，会减少叠加粗粒度位置信息，模型无法学习到宏观的位置信息，如果值过大则，频率就会减少，很多维度变化会趋近与0，感知不到位置信息。$2i/d_{\text{model}}$是为了保证频率在维度上均匀分布

Q: 为什么位置编码可以简单和embedding相加？

A: 原因如下：
1. 保证了输入的维度
2. 有利于反向传播： 梯度下降计算方便
3. 信息叠加：同时保留了位置和编码信息
4. 简单原则：神经网络架构设计的理念就是选择最简单、最不会破坏原有信息的方式进行信息融合